# Smart Care Analytics

## Packages 

In [ ]:
import pdfplumber
import pandas as pd
import re
import os
import openpyxl
import numpy as np
import matplotlib.pyplot as plt
import csv


In [11]:
# --- CONFIGURATION ---
# Remplacez par le nom exact de votre fichier. 
# Note : Le fichier fourni est "SLP-CHF2012.pdf", mais vous avez mentionné 2021.
PDF_PATH = "fichier/SLP-CHF2012.pdf" 
OUTPUT_EXCEL = "Rapport_Donnees_Extraites.xlsx"

In [ ]:
def clean_text(text):
    """Nettoie le texte : enlève les retours à la ligne et les espaces doubles."""
    if text:
        text = text.replace('\n', ' ').strip()
        return text
    return ""

def clean_numeric(val):
    """Formatte les nombres pour la base de données (ex: '2 132' -> 2132)."""
    if not val: return 0
    # On garde les chiffres, les points et les virgules, on enlève le reste
    clean_val = re.sub(r'[^\d.,-]', '', str(val))
    clean_val = clean_val.replace(',', '.') # Standardisation du séparateur décimal
    try:
        return float(clean_val) if '.' in clean_val else int(clean_val)
    except:
        return val

def extraire_par_section(pdf_path, output_csv):
    all_data = []
    
    print(f"Début de l'extraction séquentielle : {pdf_path}")
    
    with pdfplumber.open(pdf_path) as pdf:
        # Lecture "petit à petit" (page par page)
        for i, page in enumerate(pdf.pages):
            print(f"Analyse de la section/page {i+1}...")
            
            tables = page.extract_tables()
            
            for table_idx, table in enumerate(tables):
                # Transformation en DataFrame pour le nettoyage
                df = pd.DataFrame(table)
                
                # 1. Nettoyage des textes (en-têtes et cellules)
                df = df.applymap(clean_text)
                
                # 2. Gestion des en-têtes (si la 1ère ligne est remplie, on l'utilise)
                if not df.empty:
                    df.columns = [f"Col_{j}" if not c else c for j, c in enumerate(df.iloc[0])]
                    df = df[1:] # On enlève la ligne d'en-tête des données
                
                # 3. Ajout d'une colonne de contexte pour la base de données
                df['source_page'] = i + 1
                df['section_id'] = f"P{i+1}_T{table_idx+1}"
                
                all_data.append(df)

    # Fusion de tous les tableaux extraits
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        
        # Suppression des lignes vides et des doublons
        final_df = final_df.dropna(how='all').drop_duplicates()
        
        # Export au format CSV (séparateur point-virgule pour Excel/SQL)
        final_df.to_csv(output_csv, index=False, sep=';', encoding='utf-8-sig', quoting=csv.QUOTE_ALL)
        print(f"--- Extraction terminée ! ---")
        print(f"Fichier disponible : {output_csv}")
    else:
        print("Aucun tableau trouvé dans le document.")

# Lancement du script
if __name__ == "__main__":
    extraire_par_section("SLP-CHF2012.pdf", "base_donnees_hospitaliere.csv")

In [13]:

# --- EXÉCUTION DU SCRIPT ---
if __name__ == "__main__":
    if os.path.exists(PDF_PATH):
        # 1. Extraction
        dfs_bruts = extraire_tableaux_pdf(PDF_PATH)
        
        # 2. Suppression des doublons et Formatage
        dfs_propres = supprimer_doublons_et_formater(dfs_bruts)
        
        # 3. Sauvegarde
        sauvegarder_excel(dfs_propres, OUTPUT_EXCEL)
    else:
        print(f"Erreur : Le fichier {PDF_PATH} est introuvable.")

--- Démarrage de l'extraction sur : fichier/SLP-CHF2012.pdf ---
Traitement de la page 1...
Traitement de la page 2...
Traitement de la page 3...
Traitement de la page 4...
--- Sauvegarde de 14 tableaux uniques dans Rapport_Donnees_Extraites.xlsx ---
Terminé avec succès !
